# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
%pip install elasticsearch
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [4]:
import json
os.chdir('../../')
dataset_path = os.path.join(os.getcwd(), 'Phase_1', 'assets', 'IR_data_news_12k.json')
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [5]:
index_name = 'information_retrieval-index'
es = Elasticsearch("http://localhost:55000")
# es.indices.create(index = index_name)

### Check the Cluster Status

In [6]:
dict(es.info())

{'name': 'c18ef7b5bf68',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'AiCt0iMnRkiYkVFamoU1gg',
 'version': {'number': '8.2.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'b174af62e8dd9f4ac4d25875e9381ffe2b9282c5',
  'build_date': '2022-04-20T10:35:10.180408517Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [7]:
# for i in tqdm(range(len(data))):
#     es.index(index = index_name, id=i, document=data[str(i)])
#     i = i + 1

###  Bulk API

In [8]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': 'ta3_index',
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [9]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 5.00 seconds


### check index

In [10]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [19]:
query= {
        "bool": {
          "should": [
              { 
                  "match": {"content" : ""}
              }, 
              
              { 
                  "match_phrase": {"content" : "محمدباقر قالیباف"}
              },
   
          ],
          "must_not": [
              {
                  "match": {"content" : "مجلس"}    
              }
          ]
        }
    }
query2= {
        "bool": {
          "should": [
              { 
                  "match": {"content" : "اورشلیم"}
              }
   
          ],
          "must_not": [
              {
                  "match": {"content" : "صهیونیست"}    
              }
          ]
        }
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [20]:
res = es.search(index=index_name, query=query, _source= ["url"])
res = dict(res)

res2 = es.search(index=index_name, query=query2, _source= ["url"])
res2 = dict(res2)

### Results

In [21]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

3 results in 0.021 s: 
https://www.farsnews.ir/news/14000822000714/نشست-سران-قوا-برگزار-شد-تداوم-تلاش-هماهنگ-سه-قوه-برای-بهبود-معیشت
https://www.farsnews.ir/news/14000815000699/تاکید-سران-قوا-بر-همکاری-هر-چه-بیشتر-برای-حل-مسائل-و-مشکلات-معیشتی
https://www.farsnews.ir/news/14001124000825/واقعیت-یک-فایل-صوتی-ماجرای-شرکت-یاس-و-فساد-سیف-و-شریفی-چه-بود


In [18]:
print("{} results in {} s: ".format(res2['hits']['total']['value'] ,res2['took']/1000))
for doc in res2['hits']['hits']:
    print(doc['_source']['url'])

0 results in 0.003 s: 
